# LangGraph ReAct Agent 이해하기

이 노트북에서는 LangGraph를 사용한 ReAct Agent의 구조와 동작 원리를 단계별로 설명합니다.

## 1. LangGraph란?

LangGraph는 LLM(Large Language Model) 애플리케이션을 **그래프 구조**로 구성할 수 있게 해주는 라이브러리입니다.

### 핵심 개념

- **Node (노드)**: 실제 작업을 수행하는 함수 (예: LLM 호출, 도구 실행)
- **Edge (엣지)**: 노드 간의 연결, 데이터 흐름을 정의
- **State (상태)**: 그래프 전체에서 공유되는 데이터
- **Conditional Edge (조건부 엣지)**: 조건에 따라 다른 노드로 분기

## 2. ReAct 패턴이란?

ReAct는 **Re**asoning + **Act**ing의 약자로, AI가 다음을 반복하는 패턴입니다:

1. **Reasoning (추론)**: 현재 상황을 분석하고 다음 행동을 결정
2. **Acting (행동)**: 도구를 사용하여 실제 작업 수행
3. **Observation (관찰)**: 행동 결과를 확인
4. **반복**: 목표 달성까지 1-3 반복

```
┌─────────────────────────────────────────────────┐
│                                                 │
│   사용자 질문                                    │
│        ↓                                        │
│   ┌─────────┐     도구 호출 필요      ┌───────┐  │
│   │  Agent  │ ──────────────────────→ │ Tools │  │
│   │  (LLM)  │ ←────────────────────── │       │  │
│   └─────────┘     결과 반환           └───────┘  │
│        ↓                                        │
│   도구 호출 불필요 (답변 완성)                    │
│        ↓                                        │
│   최종 응답                                      │
│                                                 │
└─────────────────────────────────────────────────┘
```

## 3. 환경 설정

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경변수 로드
load_dotenv()

# API 키 확인
api_key = os.getenv("GOOGLE_API_KEY")
print(f"API Key 설정됨: {'예' if api_key else '아니오'}")

## 4. State (상태) 정의

State는 그래프의 모든 노드에서 공유되는 데이터입니다.

이 프로젝트에서는 `messages` 리스트 하나만 사용합니다.

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
    """에이전트의 상태를 정의하는 TypedDict"""
    
    # messages: 대화 기록을 저장하는 리스트
    # add_messages: 새 메시지를 기존 리스트에 추가하는 reducer 함수
    messages: Annotated[list, add_messages]

# add_messages의 동작 예시
print("add_messages는 메시지를 누적시킵니다:")
print("기존: [msg1, msg2] + 새로운: [msg3] = [msg1, msg2, msg3]")

## 5. Tools (도구) 정의

도구는 LLM이 호출할 수 있는 함수입니다. `@tool` 데코레이터를 사용하여 정의합니다.

In [ ]:
from langchain_core.tools import tool

@tool
def search_web(query: str) -> str:
    """웹에서 정보를 검색합니다.
    
    Args:
        query: 검색할 쿼리 문자열
    
    Returns:
        검색 결과 문자열
    """
    # 실제로는 API를 호출하지만, 여기서는 시뮬레이션
    query_lower = query.lower()
    
    if "langgraph" in query_lower:
        return "LangGraph는 LLM 애플리케이션을 그래프로 구성하는 라이브러리입니다."
    elif "react" in query_lower:
        return "ReAct는 Reasoning과 Acting을 결합한 AI 에이전트 패턴입니다."
    else:
        return f"'{query}'에 대한 검색 결과가 없습니다."

@tool
def calculator(expression: str) -> str:
    """수학 표현식을 계산합니다.
    
    Args:
        expression: 계산할 수학 표현식 (예: "2 + 2")
    
    Returns:
        계산 결과
    """
    # 안전한 문자만 허용
    allowed_chars = set("0123456789+-*/(). ")
    if not all(c in allowed_chars for c in expression):
        return "오류: 유효하지 않은 문자가 포함되어 있습니다."
    
    try:
        result = eval(expression)
        return f"결과: {result}"
    except Exception as e:
        return f"계산 오류: {e}"

# 도구 리스트
tools = [search_web, calculator]

# 도구 정보 확인
for t in tools:
    print(f"도구 이름: {t.name}")
    print(f"도구 설명: {t.description}")
    print("---")

## 6. LLM 설정

Google Gemini 모델을 사용하며, 도구를 바인딩합니다.

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

# LLM 인스턴스 생성
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",      # 사용할 모델
    google_api_key=api_key,        # API 키
    temperature=0,                  # 0 = 결정적(항상 같은 답변)
)

# 도구를 LLM에 바인딩
# 이렇게 하면 LLM이 도구를 "호출"할 수 있게 됩니다
llm_with_tools = llm.bind_tools(tools)

print("LLM이 사용 가능한 도구:")
for t in tools:
    print(f"  - {t.name}: {t.description[:50]}...")

## 7. 그래프 노드 정의

### Agent 노드
LLM을 호출하고 응답을 생성합니다.

In [ ]:
from langchain_core.messages import AIMessage

def agent_node(state: AgentState) -> dict:
    """LLM을 호출하여 응답을 생성하는 노드
    
    Args:
        state: 현재 에이전트 상태 (messages 포함)
    
    Returns:
        새 메시지가 포함된 상태 업데이트
    """
    # 현재까지의 모든 메시지를 LLM에 전달
    response = llm_with_tools.invoke(state["messages"])
    
    # 새 메시지를 반환 (add_messages reducer가 기존 리스트에 추가)
    return {"messages": [response]}

print("agent_node: LLM을 호출하고 응답을 messages에 추가")

### 라우터 함수
다음에 어떤 노드로 갈지 결정합니다.

In [ ]:
from typing import Literal
from langgraph.graph import END

def should_continue(state: AgentState) -> Literal["tools", "__end__"]:
    """도구를 호출할지, 종료할지 결정하는 라우터 함수
    
    Args:
        state: 현재 에이전트 상태
    
    Returns:
        "tools": LLM이 도구 호출을 요청한 경우
        "__end__": 도구 호출 없이 응답이 완성된 경우
    """
    # 마지막 메시지 가져오기
    last_message = state["messages"][-1]
    
    # AIMessage이고 tool_calls가 있으면 도구 실행으로 분기
    if isinstance(last_message, AIMessage) and last_message.tool_calls:
        print(f"  → 도구 호출 감지: {[tc['name'] for tc in last_message.tool_calls]}")
        return "tools"
    
    # 도구 호출이 없으면 종료
    print("  → 도구 호출 없음, 종료")
    return END

print("should_continue: tool_calls 유무에 따라 분기 결정")

## 8. 그래프 구성

노드와 엣지를 연결하여 완전한 그래프를 만듭니다.

In [ ]:
from langgraph.graph import StateGraph
from langgraph.prebuilt import ToolNode

# 1. 그래프 생성 (상태 타입 지정)
graph = StateGraph(AgentState)

# 2. 노드 추가
graph.add_node("agent", agent_node)      # LLM 호출 노드
graph.add_node("tools", ToolNode(tools)) # 도구 실행 노드 (LangGraph 내장)

# 3. 시작점 설정
graph.set_entry_point("agent")

# 4. 조건부 엣지 추가 (agent → tools 또는 END)
graph.add_conditional_edges(
    "agent",           # 출발 노드
    should_continue,   # 라우터 함수
    {
        "tools": "tools",  # "tools" 반환 시 tools 노드로
        END: END,          # END 반환 시 종료
    }
)

# 5. 일반 엣지 추가 (tools → agent)
graph.add_edge("tools", "agent")  # 도구 실행 후 다시 agent로

# 6. 그래프 컴파일
agent = graph.compile()

print("그래프 구조:")
print("""
  ┌──────────────────────────────────────┐
  │           Entry Point                │
  └──────────────┬───────────────────────┘
                 ↓
  ┌──────────────────────────────────────┐
  │           agent_node                 │
  │     (LLM 호출, 응답 생성)              │
  └──────────────┬───────────────────────┘
                 ↓
  ┌──────────────────────────────────────┐
  │         should_continue              │
  │     (tool_calls 있는지 확인)           │
  └───────┬────────────────┬─────────────┘
          ↓                ↓
    tool_calls 있음    tool_calls 없음
          ↓                ↓
  ┌───────────────┐  ┌─────────────┐
  │  tools_node   │  │     END     │
  │ (도구 실행)    │  │  (최종 응답) │
  └───────┬───────┘  └─────────────┘
          │
          └──────→ agent_node (다시 루프)
""")

## 9. 에이전트 실행 예제

실제로 에이전트를 실행해봅니다.

In [ ]:
from langchain_core.messages import HumanMessage

def run_agent(query: str):
    """에이전트를 실행하고 결과를 출력"""
    print(f"\n{'='*50}")
    print(f"질문: {query}")
    print('='*50)
    
    # 초기 상태 생성
    initial_state = {
        "messages": [HumanMessage(content=query)]
    }
    
    # 에이전트 실행
    result = agent.invoke(initial_state)
    
    # 최종 응답 출력
    final_message = result["messages"][-1]
    print(f"\n최종 응답: {final_message.content}")
    
    return result

### 예제 1: 웹 검색이 필요한 질문

In [ ]:
result1 = run_agent("LangGraph가 무엇인지 검색해줘")

### 예제 2: 계산이 필요한 질문

In [ ]:
result2 = run_agent("25 * 4 + 10을 계산해줘")

### 예제 3: 도구가 필요 없는 질문

In [ ]:
result3 = run_agent("안녕하세요!")

## 10. 메시지 흐름 상세 분석

각 단계에서 messages가 어떻게 변하는지 확인합니다.

In [ ]:
def run_agent_verbose(query: str):
    """에이전트를 실행하며 각 단계의 메시지를 출력"""
    print(f"\n{'='*60}")
    print(f"질문: {query}")
    print('='*60)
    
    initial_state = {
        "messages": [HumanMessage(content=query)]
    }
    
    # stream으로 각 단계 확인
    for i, step in enumerate(agent.stream(initial_state)):
        print(f"\n--- Step {i+1} ---")
        
        for node_name, node_output in step.items():
            print(f"노드: {node_name}")
            
            for msg in node_output.get("messages", []):
                msg_type = type(msg).__name__
                
                if hasattr(msg, 'tool_calls') and msg.tool_calls:
                    print(f"  [{msg_type}] 도구 호출: {msg.tool_calls}")
                elif hasattr(msg, 'content'):
                    content = msg.content[:100] + "..." if len(str(msg.content)) > 100 else msg.content
                    print(f"  [{msg_type}] {content}")

# 계산 예제로 흐름 확인
run_agent_verbose("100 / 4를 계산해줘")

## 11. 핵심 정리

### LangGraph의 핵심 구성 요소

| 구성 요소 | 역할 | 이 프로젝트에서 |
|----------|------|---------------|
| **State** | 그래프 전체에서 공유되는 데이터 | `AgentState` (messages 리스트) |
| **Node** | 실제 작업을 수행하는 함수 | `agent_node`, `ToolNode` |
| **Edge** | 노드 간 연결 | `tools` → `agent` |
| **Conditional Edge** | 조건에 따른 분기 | `should_continue` 함수 |

### ReAct 루프 동작 방식

1. **사용자 질문** → `messages`에 `HumanMessage` 추가
2. **agent_node** → LLM이 질문을 분석하고 응답 또는 도구 호출 결정
3. **should_continue** → `tool_calls`가 있으면 `tools`, 없으면 `END`
4. **tools_node** → 도구 실행, 결과를 `messages`에 추가
5. **다시 agent_node** → 도구 결과를 바탕으로 최종 응답 생성
6. **END** → 최종 응답 반환

## 12. 다음 단계

이 기본 구조를 확장하여:

- 더 많은 도구 추가 (실제 웹 검색 API, 데이터베이스 조회 등)
- 대화 기록 저장 (메모리 기능)
- 스트리밍 응답
- 에러 핸들링 개선
- 여러 에이전트 조합 (멀티 에이전트 시스템)